# Recurrent Neural Networks

Recurrent Neural Networks (RNNs) sind neben den CNNs eine weitere spezielle Form von Neuronalen Netzen. 
RNNs werden hauptsächlich für Sequenzen, die in einer festen Reihenfolge angeordnet sind, verwendet. In diesen Fällen ist die Reihenfolge der einzelnen Elemente der Sequenz oft entscheidend für die Interpretation der gesamten Sequenz.

Sprachen als klassisches Beispiel bieten sich sofort an. Das liegt daran, dass der Inhalt eines Satzes die Interpretation der einzelnen Wörter beeinflusst. 

Ein Beispiel:

> Ich bin kein Fan von diesem Film.


Das Wort "*Fan*" hat eine positive Konnotation. Aber das "*nicht*" vor dem Wort, dreht die Interpretation um. Das heißt, das Wort "*Fan*" sollte im Kontext des gesamten Satzes interpretiert werden. 
RNNs können aber auch in der Chemie/Pharmazie eingesetzt werden. So eignen sich zum Beispiel Smiles `strings` oder Proteinsequenzen für RNNs. 

 `()` haben einen starken Einfluss darauf, wie einzelne Teile des Smiles interpretiert werden können.

<div align="center">

`CCCC`|`CC(C)C`
------|--------
<img align="center" src="Img/rnn/mol1.png" width="200"/> | <img  align="center" src="Img/rnn/mol2.png" width="200"/> 

</div>


Das allgemeine Konzept eines RNN ist relativ einfach:
Wort für Wort (oder auch Zeichen für Zeichen) wird ein Satz (Smiles) durch das Netz geführt. 
Die Outputlayer wird zunächst komplett ignoriert, aber nachdem ein Wort durch das Netz gelaufen ist, werden die Activations der versteckten Schicht ($h_1$) gespeichert.

Anhand des Beispielsatzes "*Hallo Welt*" wird dies im Bild erklärt. $h_1$ sind hier die Activations für das Wort "Hallo".

Im Zusammenhang mit RNNs bezeichnen wir die Activations der Hidden Layer auch als **Hidden State**. $h_1$ ist der Hidden State für das Wort "*Hallo*".

Als nächstes wird das zweite Wort durch das Netz geschickt. Wir wollen $h_2$ berechnen, aber zu den regulären Activations des Wortes "Welt" addieren wir auch die Aktivierungen $h_1$. $h_2$ ist also eine Kombination aus den Activations von "Welt", aber auch von "Hallo". Das bedeutet, dass das Wort "Welt" zusammen mit dem vorangegangenen Wort interpretiert wurde.


<div align="center">
<img align="center" src="Img/rnn/rnn_1.svg.png" width="200"/> 
</div>

Hätten wir ein drittes Wort, würde $h_3$ aus den Activations des dritten Wortes und $h_2$ berechnet werden. Und da $h_2$ die Informationen sowohl des zweiten als auch des ersten Wortes enthält, beeinflussen beide Wörter die Interpretation des dritten Wortes.


<div align="center">
<img align="center" src="https://miro.medium.com/max/724/1*1U8H9EZiDqfylJU7Im23Ag.gif">
    
*Source: Michael Phi - An illustrated Guide to Recurrent Neural Networks*

</div>
    
Im GIF sehen Sie, dass der Einfluss des Hidden States von "*What*" (schwarz), dem ersten Wort, immer geringer wird, je näher wir dem Ende des Satzes kommen. Er hat jedoch immer noch einen Einfluss auf die Interpretation des letzten Wortes.


Der Hidden State des letzten Teils des Satzes ("*?* "), im Beispiel $O5$ ($h_5$) genannt, ist eine Kombination aus allen vorherigen Hidden States und den Activations von "*?* ".

<div align="center">
            
<img align="center" src="https://ichi.pro/assets/images/max/724/1*yQzlE7JseW32VVU-xlOUvQ.png">

</div>

Wir können diesen Hidden State als Input für ein  weiteres Netzwerk verwenden, das seine Vorhersage auf der Grundlage dieses letzten Hidden States macht.

Ähnlich wie ein CNN verwendet wird, um ein Bild in einen Vektor umzuwandeln, werden RNNs verwendet, um Sequenzen in Vektoren umzuwandeln.


# Datenaufbereitung:

Doch bevor wir unser RNN trainieren, müssen wir die Daten in das richtige Format bringen. Buchstaben und Wörter können nicht einfach von einem neuronalen Netz gelesen werden.
Wie bei den Bezeichnungen aus dem MNIST-Datensatz (0-9) können wir Wörter oder, im Fall von Smiles, Zeichen "one-hot" kodieren. 

Angenommen wir haben zwei Smiles:

`smiles = ["CCN=C=O","NC(=O)CC(=O)O"]`

Es gibt insgesamt sechs verschiedene Symbole:
`C`, `N`, `=`, `O`, `(`, `)` 

Wir können ein `C` als einen Vektor der Länge 6 darstellen. Dieser hat an der ersten Position eine `1` und sonst nur Nullen. Ein `N` können wir ebenfalls als Vektordarstellen, nur dass wir die `1` um eine Position verschieben.

Dies können wir für alle Symbole in den Smiles machen:

```python
"C" = [1,0,0,0,0,0]
"N" = [0,1,0,0,0,0]
"=" = [0,0,1,0,0,0]
"O" = [0,0,0,1,0,0]
"(" = [0,0,0,0,1,0]
")" = [0,0,0,0,0,1]
```
Diese Symbole werden auch oft **Tokens** genannt.
Wir können also einen Smiles `string` mithilfe dieser Regeln kodieren. Wir brauchen also pro Smiles eine Matrix:

```python
"CCN=C=O" -> np.array([[1,0,0,0,0,0],
                      [1,0,0,0,0,0],
                      [0,1,0,0,0,0],
                      [0,0,1,0,0,0],
                      [1,0,0,0,0,0],
                      [0,0,1,0,0,0],
                      [0,0,0,1,0,0]])
```

Der `string`  `"CCN=C=O"` wird zu einer Matrix, in der jede Zeile ein Token ist und jede Spalte angibt, welche Symbole dieser Zeile zugeordnet sind.

Mit dem folgenden Code können Sie diese Umwandlung automatisieren.
Viele Funktionen sind schon von uns vorgeschrieben. Wenn Sie aber trotzdem daran interessiert sind, wie diese Funktionen genau aussehen, finden Sie den Code in `../utils/utils.py`.

In [ ]:
import torch
from torch import nn, optim
from rdkit.Chem import AllChem as Chem
import numpy as np
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
import sys
from os.path import exists
if 'google.colab' in sys.modules:
    !pip install rdkit==2022.3.4
    if exists("utils.py") == False:
        !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
    %run utils.py
else:
    %run ../utils/utils.py

In [ ]:
smiles = ["CCN=C=O","NC(=O)CC(=O)O"]

Zunächst brauchen wir eine Art Wörterbuch, das alle vorkommenden Symbole speichert und ihnen eine Zahl zuweist. Diese Zahl gibt auch an, an welcher Stelle im One-Hot-Vektor die `1` stehen wird. 

In [ ]:
dictionary = create_dict(smiles)
dictionary

Sie sehen, dem `=` wird eine `0` zugewiesen und dem `N` eine `1` und so weiter....

Mit der Funktion `tokenize()` kann man die Smiles in eine Zahlenfolge umwandeln. Wir stellen nun die Smiles-`string` mit den Zahlen dar. 
Der Funktion muss lediglich mitgeteilt werden, welche Smiles kodiert werden sollen und welcher `dictionary` dafür verwendet werden soll.

In [ ]:
tokenized_smiles = tokenize(smiles,dictionary)
tokenized_smiles

Die Smiles werden nun als eine einfache Zahlenfolge dargestellt.
Diese sind jedoch immer noch unterschiedlich lang.

In [ ]:
[len(x) for x in tokenized_smiles]

Der erste Smiles besteht aus 7 Symbolen/Tokens, der andere aus 13. Das ist ein Problem, denn ein RNN erwartet, dass jede Sequenz gleich lang ist. Das ist natürlich nicht immer möglich, denn größere Moleküle haben mehr Symbole als kleinere. 
Um das Problem zu lösen, *"padden"* wir alle Sequenzen auf die Länge des längsten Smiles.
Das "*padden*" bedeutet, dass wir einen neuen Token zu unserem Wörterbuch hinzufügen: `"<pad>"`. Dieses Symbol wird zu jedem Smiles-`string` hinzugefügt, bis sie dieser die gleiche Länge hat wie der längste Smiles. 
Das `"<pad>"` soll dem Netz mitteilen, dass diese Symbole für den eigentlichen Smiles nicht mehr relevant sind.

In [ ]:
max_smiles_length = max([len(x) for x in tokenized_smiles])
max_smiles_length

In [ ]:
dictionary["<pad>"] = len(dictionary)
dictionary

Jetzt haben wir den Token `<pad>` zu unserem Wörterbuch hinzugefügt. Das Letzte, was wir tun müssen, ist, diesen Token an unseren ersten Smiles `tokenized_smiles[0]` anzuhängen.

In [ ]:
num_fehlende_tokens = max_smiles_length-len(tokenized_smiles[0])
tokenized_smiles[0] += [dictionary["<pad>"]] * num_fehlende_tokens 
tokenized_smiles[0]

Jetzt sind beide Smiles gleich lang:

In [ ]:
[len(x) for x in tokenized_smiles]

Da die Smiles jetzt die gleiche Länge haben, können wir nun die Zahlen in One-Hot kodierte Vektoren umwandeln.

In [ ]:
vocabulary_length = len(dictionary)
print(vocabulary_length)

Insgesamt befinden sich 7 Symbole in unserem Wörterbuch.
Mit der Funktion `token_to_onehot` werden aus den `tokenized_smiles` Matrizen.

In [ ]:
onehot_tokens = token_to_onehot(tokenized_smiles, vocabulary_length)
print(onehot_tokens[0])
      
print(onehot_tokens.shape)

`onehot_tokens` ist `np.array` mit den Dimensionen  `(2,13,7)` . Die erste Dimension ist die Anzahl der Smiles (`2`). Die zweite Dimension ist die Länge der Sequenzen (`13`). Die dritte Dimension ist die Anzahl der verschiedenen Token (`7`).

An sich wären unsere Daten jetzt bereit für ein RNN. Aber anstatt diese One-Hot kodierten Vektoren als Input zu nehmen, verwenden wir zunächst eine *Embedding Layer*. 

# Word Embeddings

Mittlerweile werden diese One-Hot kodierten Vektoren nicht mehr direkt als Input verwendet. Bevor sie in das Netz eingespeist werden, wird eine Embedding Layer verwendet. Diese ersetzt die One-Hot-codierten Vektoren durch zunächst zufällige Zahlen. Um besser zu verstehen, was gemeint ist, schauen wir uns zunächst eine Embedding Layer an.

In [ ]:
np.random.seed(1234)
embedding_layer=np.random.rand(7,4)
embedding_layer

Eine Embedding Layer besteht aus einer einzigen Weight Matrix. Diesen enthält zufällige Zahlen. Die Anzahl der Zeilen entspricht genau der Anzahl der verschiedenen Token in unserem Wörterbuch. 
Eine Embedding Layer tauscht, einfach den Vektor `[1,0,0,0,0,0,0]` mit der ersten Reihe aus der `embedding_layer` aus. Welcher `embedding_layer[0,:]= [0.19151945, 0.62210877, 0.43772774, 0.78535858]` ist.

Um das zu erreichen, müssen wir einfach die One-Hot kodierten Smiles mit der Embedding Layer multiplizieren:

In [ ]:
token_embeddings = np.matmul(onehot_tokens,embedding_layer)
print(token_embeddings[0])

Sie können die Embeddings des ersten Smiles hier oben sehen.
Unten sehen Sie die erste Zeile des One-Hot kodierten Smiles.

In [ ]:
onehot_tokens[0,0,:]

Wenn Sie sich jetzt in der Weight Matrix der `embedding_layer` die dritte Reihe (index `2`) anschauen, fällt auf, dass dieser Vektor genau dieselben Werte hat wie die erste Reihe in der `token_embeddings` Layer.

In [ ]:
embedding_layer[2,:]

In [ ]:
token_embeddings[0,0,:]

Einfacher erklärt: 
Eine Embedding Layer wandelt One-Hot kodierte Vektoren in Vektoren mit zufälligen Weights um. 

*Aber warum wird das gemacht?

Ein Vorteil ist, dass Texte oder sogar Smiles in den meisten Fällen aus mehr als nur 7 Symbolen oder Wörtern bestehen. Würden wir zum Beispiel alle Wörter kodieren, die in einem Dokument vorkommen, würden diese Vektoren sehr lang werden. Durch die "Embedding" der Vektoren können wir zunächst die Größe dieser Inputvektoren verringern.

Noch wichtiger ist, dass die Weights in der Embedding Layer erlernt werden können. Das bedeutet, dass diese Weights während der Backpropagation geupdatet werden.
So passen sich die Embeddings während des Trainings an. Das ist praktisch, denn man erwartet, dass ähnliche Wörter nach dem Training ähnliche Embeddings erhalten. Zum Beispiel sind die Wörter LKW und Auto im Gebrauch ähnlicher als Auto und Strand. 
Wenn Auto und Lkw ähnliche Embeddings haben, d. h. durch ähnliche Vektoren beschrieben werden, dann können sie im Kontext des Satzes leichter verarbeitet werden.


> Ein Auto fährt auf der Straße

> Ein LKW fährt auf der Straße

Beschreiben zwei sehr ähnlich Situation und wenn sich auch die numerischen Repräsentationen ähneln, fällt es dem Netzwerk leichter diese zu lernen.


Im Falle von Smiles kann man argumentieren, dass die Rolle eines Stickstoffs in einem Molekül eher der eines Kohlenstoffs als der eines Fluors entspricht. Dies sollte sich insbesondere in den Embeddings widerspiegeln.


# RNNs

Wir haben jetzt die Smiles in das richtige Format umgewandelt. Wir müssen nur noch das `np.array` in einen Tensor umwandeln. Achten Sie darauf, dass wir zusätzlich die Funktion `.permute` verwenden. Die Funktion `.permute` wird verwendet, um Dimensionen eines Tensors zu tauschen. Das ist notwendig, da PyTorch bei RNNs erwartet, dass der Tensor wie folgt angeordnet ist:
`[Länge des Smiles, Anzahl der Smiles, Embeddinggröße]`

In [ ]:
token_embeddings_tensor = torch.tensor(token_embeddings, dtype= torch.float).permute(1,0,2)
token_embeddings_tensor.shape

Der Tensor `token_embeddings_tensor` hat die oben genannten Dimensionen. Jeder Smiles besteht aus `13` Token, unser Batch besteht aus `2` Smiles und jeder Token wird durch `4` Werte beschrieben. 

Wir können nun ein RNN definieren. Wie üblich gibt es auch im Modul `torch.nn` ein RNN.
Auch hier muss man bei der Definition der Dimensionen vorsichtig sein. Die erste Dimension ist die Größe Inputvektoren, also die Embeddinggröße (`4`). Die zweite Dimension gibt an, wie viele Nodes wir in der Hidden Layer haben wollen. Damit wird auch festgelegt, wie groß die Vektoren des Hidden States sein sollen.


In [ ]:
torch.manual_seed(1234)
rnn = nn.RNN(4,10)

Sie können jetzt einfach den `token_embeddings_tensor` durch das `rnn` führen.

In [ ]:
output_rnn = rnn(token_embeddings_tensor)
len(output_rnn)

Der Output des RNN (`output_rnn`) ist eine Liste mit der Länge zwei.
Wir schauen uns zunächst das erste Objekt des Outputs an.

In [ ]:
print(output_rnn[0])

In [ ]:
print(output_rnn[0].shape)

Der Output `output_rnn[0]` hat die Dimensionen `[13, 2, 10]`. Das einzige, was sich im Vergleich zum Input geändert hat, ist die letzte Dimension. Statt der Dimension `4` ist es jetzt `10`. 

Tatsächlich enthält der erste Teil des RNN Outputs die Hidden States jedes Symbols im Smiles.

Denken Sie an das GIF zurück:

<div align="center">
<img  align="center" src="https://miro.medium.com/max/724/1*1U8H9EZiDqfylJU7Im23Ag.gif">
    
*Quelle: Michael Phi - An illustrated Guide to Recurrent Neural Networks.*

</div>
    
`output_rnn[0]` enthält $O1$ bis $O5$. Da unsere Sequenzen aber die Länge 13 haben, enthält `output_rnn[0]` 13 Hidden States.

Aber was enthält `output_rnn[1]`?

In [ ]:
output_rnn[1]

In [ ]:
output_rnn[1].shape

`output_rnn[1]` enthält NUR den letzten Hidden State. Im GIF ist das $O5$, bei uns wäre es $O13$. Dieser Hidden State beschreibt (theoretisch) die komplette Sequenz und ist daher besonders wichtig.

Das `output_rnn[0][-1]== output_rnn[1][0]` kann man auch kontrollieren:

In [ ]:
print(output_rnn[0][-1])
output_rnn[1]


Um genauer zu verstehen, was passiert, werden wir das PyTorch RNN selbst neu programmieren.


Nehmen wir an, wir haben einen Satz `Satz = ["Hallo", "Welt"]`. Wir haben diesen als zwei Wörter in einer Liste gespeichert. 

Wir definieren auch zwei einfache lineare Layers.  Die eine mappt den Input von Embeddinggröße `4`  auf `10` Dimensionen. Die andere Layer mappt von `10` auf `10` DImensionen.

Durch das erste Netzwerk schicken wir das erste Wort `satz[0]` und speichern den Hidden State in `output_1`.


```python
satz = ["Hello", "World"]

lin_1 = nn.Linear(4,10) 

lin_2 = nn.Linear(10,10)

output_1 =rnn(satz[0])
```
Als Nächstes führen wir auch das zweite Wort „World“ durch das `lin_1`. Doch im Anschluss addieren wir auch den `lin_2(output_1)` dazu. 

```python
satz = ["Hello", "World"]

lin_1 = nn.Linear(4,10) 

lin_2 = nn.Linear(10,10)

output_1 = lin_1(satz[0])

output_2 = lin_1(satz[1]) + lin_2(output_1)
```

Das heißt, der Hidden State `output_2` wird nicht alleine durch das Wort `"World"` bestimmt, sondern der Hidden State zuvor hat auch Einfluss. Tatsächlich fügen wir auch noch eine nicht-lineare Aktivierungsfunktion hinzu. In RNNs wird per Default eine Tanh-Funktion anstatt einer ReLU-Funktion benutzt.

```python
satz = ["Hello", "World"]

lin_1 = nn.Linear(4,10) 

lin_2 = nn.Linear(10,10)

output_1 = lin_1(satz[0])

output_2 = torch.tanh(lin_1(satz[1]) + lin_2(output_1))
```

Hätten wir noch ein drittes Wort im Satz (`satz[2]`), dann würde sich der Schritt wiederholen. Wir addieren diesmal, aber nicht `output_1`, sondern `output_2` hinzu:

```python
satz = ["Hello", "World", "Dude"]

lin_1 = nn.Linear(4,10) 

lin_2 = nn.Linear(10,10)

output_1 = lin_1(satz[0])

output_2 = torch.tanh(lin_1(satz[2]) + lin_2(output_1))

output_3 = torch.tanh(lin_1(satz[3]) + lin_2(output_2))

```


Um dies zu kontrollieren, schreiben wir ein eigenes Programm dafür.
Zuerst speichern wir die Weights der `rnn`. Diese können wir nun selbst verwenden.
Denken Sie daran, dass `nn.Linear()` nichts anderes ausführt als: `torch.mm(X,W.t())+b`.

In [ ]:
w_1=list(rnn.parameters())[0]
w_2=list(rnn.parameters())[1]
b_1=list(rnn.parameters())[2]
b_2=list(rnn.parameters())[3]

Mit diesen Weights können Sie nun den Hidden State für das erste Symbol in der Smiles Sequenz berechnen (`lin_1`). Diese befinden sich in `token_embeddings_tensor[0]`.

In [ ]:
activations_jetzt = torch.mm(token_embeddings_tensor[0],____)+____
activations_jetzt

<details>
    <summary><b>Lösung:</b></summary>

```python
activations_jetzt = torch.mm(token_embeddings_tensor[0],w_1.t())+b_1
activations_jetzt
```
</details>

Als nächstes transformieren wir den Hidden State des vorherigen Tokens (`lin_2`). 
Allerdings befinden wir uns im Moment beim ersten Wort/Token. Wir haben also noch keinen Hidden States eines vorherigen Tokens. Dieser Teil wurde im bisherigen Text ausgelassen. Tatsächlich beginnen wir mit einem Hidden State, in dem alle Werte Null sind. `h0 = torch.zeros(2,10)`

In [ ]:
h0 = torch.zeros(2,10)

activations_vorher = torch.mm(___,____)+____

<details>
    <summary><b>Lösung:</b></summary>

```python
h0 = torch.zeros(2,10)

activations_vorher = torch.mm(h0,w_2.t())+b_2
```
</details>

Im letzten Schritt werden die beiden Activations addiert und eine `torch.tanh` Aktivierungsfunktion angewandt.

In [ ]:
torch.tanh(___________+_____________)

<details>
    <summary><b>Lösung:</b></summary>

```python
torch.tanh(activations_jetzt+activations_vorher)
```
</details>

Dies ist der Hidden State für den erste Token des Smilea.
Wir können diesen auch mit dem Hidden State vom`nn.RNN`  vergleichen und sehen, dass diese identisch sind.

In [ ]:
output_rnn[0][0]

Wir wollen die Hidden States nicht nur für den ersten Token berechnen, sondern für alle Tokens im Smiles. Daher benötigen wir einen `for-loop`. 

Zuerst initialisieren wir den ersten Hidden States mit Nullen. Und dann schreiben wir einen `for-loop`, der alle 13 Tokens durchläuft.

In [ ]:
h0 = torch.zeros(2,10)
for i in range(max_smiles_length):
    activations_jetzt =  # achten Sie bei der Berechnung darauf immer das i Element aus den Input auszuwählen
    activations_vorher = 
    h0 = torch.tanh(activations_jetzt+activations_vorher) # <-- Der output wird als h0 gespeichert, 
h0                                                        #     um ihn in der nächsten Iteration als neues h0
                                                          #     zuverwenden              

<details>
    <summary><b>Lösung:</b></summary>

```python
h0 = torch.zeros(2,10)
for i in range(max_smiles_length):
    activations_jetzt = torch.mm(token_embeddings_tensor[i],w_1.t())+b_1
    activations_vorher = torch.mm(h0,w_2.t())+b_2
    h0 = torch.tanh(activations_jetzt+activations_vorher) 
h0                                                     
```                                                          
</details>

<details>
    <summary><b></b></summary>

```python
h0 = torch.zeros(2,10)
for i in range(max_smiles_length):
    activations_jetzt = torch.mm(token_embeddings_tensor[i],w_1.t())+b_1
    activations_vorher = torch.mm(h0,w_2.t())+b_2
    h0 = torch.tanh(activations_jetzt+activations_vorher) 
h0                                                     
```                                                          
</details>

`h0` enthält nun den finalen Hidden State. Auch hier können wir überprüfen, ob unser Ergebnis mit dem von PyTorch `nn.RNN` identisch ist.

In [ ]:
output_rnn[1]

Natürlich ist es einfacher, die vorgeschriebene Funktion von PyTorch zu verwenden. 
Aber sie selbst zu programmieren sollte Ihnen helfen, besser zu verstehen, was genau in einem RNN passiert.

Außerdem veranschaulicht der Code die größte Schwäche von RNNs: der `for-loop`.
Wir können einen Satz/Smiles nicht auf einmal durch das Netzwerk führen. 
Jedes Wort/Symbol muss eines nach dem anderen durch das Netzwerk gegeben werden. Das macht RNNs extrem langsam.


# PyTorch RNN

PyTorch bietet uns nicht nur RNNs, sondern auch `nn.Embedding` Layers. Das ist praktisch. Zum einen macht es die Backpropagation einfacher. Zusätzlich müssen wir nicht die 
One-Hot kodierten Vektoren berechnen. PyTorch nimmt als Input sofort die tokenisierten Smiles (`tokenized_smiles`) als Input. 

In [ ]:
tokenized_smiles

In [ ]:
emb = nn.Embedding(7,4, padding_idx = dictionary["<pad>"])

Hier haben wir eine `torch` Embedding Layer definiert. Sie nimmt als Input die Anzahl der verschiedenen Symbole/Token in unserem Datensatz. In unserem Fall wäre dies `7`. Der zweite Parameter gibt die Größe der Embeddingsvektoren an. Wir bleiben bei der Größe `4`. Als letztes können wir PyTorch mitteilen, welcher Token, d.h. welche Zahl für das Padding steht. PyTorch wird dann die Embeddings für diese Token auf Null setzen.

In [ ]:
emb(torch.tensor(tokenized_smiles)).shape

Der Output dieser Embedding Layer hat noch nicht das richtige Format. Wir müssen noch die Dimensionen des Tensors mit `Permute` ändern. 
Wir können all diese Schritte in ein `nn.Sequential()` Modul packen. 

*Im Pytorch `nn` Modul gibt es kein Permute, das wurde von uns so angepasst, dass es auch in `nn.Sequential` funktioniert. Deshalb brauchen wir auch kein "nn." vor dem Permute.

In [ ]:
model = nn.Sequential(nn.Embedding(7,4, padding_idx = dictionary["<pad>"]),
                     Permute(1,0,2),
                     nn.RNN(4,10))

model

Die `tokenized_smiles` können nun durch das `model` geführt werden. Mit `[1][0,:,:]` werden die finalen Hidden States im richtigen Format extrahiert. Wir können diese direkt in eine lineare Layer einfügen. Da wir den Output mit `[1][0,:,:]` indizieren müssen, können wir die linearen Layers nicht direkt in demselben `nn.Sequential()`-Modell verwenden. Wir brauchen ein zweites Modell, das `output_rnn` als Input nimmt.

In [ ]:
output_rnn= model(torch.tensor(tokenized_smiles))[1][0,:,:]

In [ ]:
pred_ll = nn.Sequential(nn.Linear(10,1))

In [ ]:
pred_ll(output_rnn)

Es gibt noch ein Problem mit dem `nn.RNN`. Im GIF kann man deutlich erkennen, dass die ersten Wörter im Satz immer weniger Einfluss haben, je länger der Satz wird. Dies kann ein Problem werden, wenn Sätze oder Smiles besonders lang werden. Vor allem, wenn Nebensätze oder im Falle von Smiles zusätzliche Branches in den `string` eingefügt werden, kann es passieren, dass der Anfang des Satzes oder desSmiles vom Netzwerk "vergessen" wird bzw. verloren geht.

Aus diesem Grund werden in der Regel komplexere RNN-Layers verwendet. Dadurch können die Netze Informationen über längere `strings` halten.

Eine beliebte Alternative ist die Gated Recurrent Unit (GRU). Das Kombinieren von Hidden States ist viel komplexer als bei "Vanilla RNNs", aber in PyTorch kann `nn.RNN` leicht durch `nn.GRU` ersetzt werden. Nichts muss am Rest des Netzwerkes geändert werden.

<div align="center">
    
RNN   |GRU
------|--------
<img align="center" src="https://miro.medium.com/max/332/0*eRJCRsikdGGu8ffA.png" width="200"/> |<img src="https://miro.medium.com/max/700/1*RiOzdOVaaeKrUotY7-1a2A.png" width="300"/> 

</div>

# Übungsaufgabe:

In der Übungsaufgabe werden wir uns einen neuen Datensatz ansehen. Der Datensatz Blut-Hirn-Schranken-Penetration (BBBP) erfasst für 2000 Moleküle, ob sie durch die Blut-Hirn-Schranke diffundieren können.

Die meisten Medikamente und Neurotransmitter können die Blut-Hirn-Schranke nicht passieren. Dies ist jedoch wichtig für Medikamente, die im zentralen Nervensystem wirken sollen. Daher ist eine genaue Vorhersage dieser Eigenschaften von großem Interesse.
Der Originaldatensatz wurde 2012 veröffentlicht. Wir verwenden jedoch einen leicht modifizierten Datensatz. Hier wurden alle Informationen zur Stereochemie aus den Smiles bereits entfernt. Außerdem enthält der Datensatz nur Smiles, die aus weniger als 75 Tokens bestehen.
> Martins, Ines Filipa, et al. “A Bayesian approach to in silico blood-brain barrier penetration modeling.” Journal of Chemical Information and Modeling 52.6 (2012): 1686-1697.



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from rdkit.Chem import AllChem as Chem
import numpy as np
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib import pyplot as plt
import sys
from os.path import exists
if 'google.colab' in sys.modules:
    !pip install rdkit==2022.3.4
    if exists("utils.py") == False:
        !wget https://raw.githubusercontent.com/kochgroup/intro_pharma_ai/main/utils/utils.py
    %run utils.py
else:
    %run ../utils/utils.py

Sie könnnen zunächst den Datensatz einlesen.

In [ ]:
data_bbbp = pd.read_csv("https://uni-muenster.sciebo.de/s/RrwUmm9mEE8wF46/download")
data_bbbp.head()

Die `smiles` werden zusammen mit dem `target` angegeben. Eine `1` bedeutet, dass diese Moleküle durch die BBB diffundieren können. In der folgenden Zelle berechnen wir den Prozentsatz der Moleküle, die diese Eigenschaft im Datensatz haben.

In [ ]:
np.sum(data_bbbp.target)/data_bbbp.shape[0]*100

Wegen des großen Ungleichgewichtes bietet sich als Metrik vor allem der ROC-AUC an.
Doch bevor wir uns dem Training zuwenden können, müssen wir erst die Daten aufbereiten.
Erstellen Sie zunächst einen `dictionary`, der allen Symbolen in den `smiles` Zahlen zuordnet.

In [ ]:
dictionary = create_dict(data_bbbp.smiles)

In [ ]:
dictionary

Mit diesem Dictionary, konvertieren Sie nun die eigentlichen Symbole der Smiles zu Zahlen.

In [ ]:
tokenized_smiles = tokenize(data_bbbp.smiles,dictionary)

Das Problem ist, wie auch schon im Beispiel, dass die Moleküle und damit die `smiles` unterschiedlich lang sind:

In [ ]:
length_ll = np.array([len(x) for x in tokenized_smiles])
length_ll

Sie müssen deswegen erst alle `tokenized_smiles` auf die gleiche Länge bringen. Und zwar auf die des längsten Smiles. 

In [ ]:
max_length = max(length_ll)
max_length

Zu allen Smiles, die aus weniger als 74 Tokens bestehen, fügen wir zusätzliche Tokens hinzu, bis sie 74 Tokens lang sind.
Der hinzugefügte Token ist `<pad>`. Wir weisen ihm den Wert `len(dictionary)` zu, da dieser die nächste unbenutzte Zahl ist.

In [ ]:
print(len(dictionary))
dictionary["<pad>"]= len(dictionary)

Der folgende Code hängt diesen Paddingtoken an alle Smiles.

In [ ]:
for i, tok_smi in enumerate(tokenized_smiles):
    tokenized_smiles[i] = tok_smi+ [dictionary["<pad>"]]*(max_length - length_ll[i])

In [ ]:
length_ll = [len(x) for x in tokenized_smiles]
length_ll

Nun sind alle `tokenized_smiles` gleich lang und im richtigen Format. Sie müssen aber zuvor wieder die Daten in Trainings- und Testdatensatz teilen. 
Dafür fügen wir die `tokenized_smiles` und Targets aus dem `data_bbbp` zusammen. 

In [ ]:
data_bbbp_tokenized = np.hstack([np.array(tokenized_smiles), data_bbbp.iloc[:,1:2]])
data_bbbp_tokenized

In [ ]:
train, test=train_test_split(data_bbbp_tokenized,test_size=0.2,train_size=0.8, random_state=1234)

Nun separieren Sie die Input und Outputs wieder von einander. Wichtig hierbei: Die `targets` befinden sich in der letzen Spalte.

In [ ]:
train_x = torch.tensor(train[:,:-1], dtype=torch.long )
train_y = torch.tensor(train[:,-1], dtype=torch.float)
test_x = torch.tensor(test[:,:-1], dtype=torch.long)
test_y = torch.tensor(test[:,-1], dtype=torch.float)

Erstellen Sie jetzt den Trainings Dataloader, damit wir mit Minibatches trainieren können. 

In [ ]:
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=32)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=32)

Jetzt definieren Sie das Model.
Wir brauchen eine Embedding Layer, eine Permute Layer und ein RNN. Hierfür verwenden wir ein GRU.

In [ ]:
torch.manual_seed(1111)
model =nn.Sequential(nn.Embedding(len(dictionary),32, padding_idx = dictionary["<pad>"]),
                     Permute(1,0,2),
                     nn.GRU(32,64))


Außerdem benötigen Sie eine lineare Layer, die Vorhersagen auf Grundlage des Outputs des GRU trifft. 
Hierfür erstellen wir ein zweites Modell mit dem Namen `pred_ll`.

Warum brauchen wir ein zweites Modell?

Das liegt daran, dass alle RNNs in PyTorch mehr als einen Output haben. Einmal alle Hidden States und einmal die finalen Hidden States. Das `nn.Sequential` Netzwerk weiß in diesem Fall nicht, welchen Output vom RNN an die lineare Layer weitergegeben werden soll.

Deshalb brauchen wir ein zweites Modell `pred_ll`. Hier verwenden wir Batchnorm und Dropout. Stellen Sie sicher, dass die Dimensionen von `BatchNorm1d` und `Linear` der Outputdimension des `GRU` entsprechen.

In [ ]:
torch.manual_seed(1111)
pred_ll = nn.Sequential(nn.BatchNorm1d(64),nn.Dropout(0.2),nn.Linear(64,1))

Auch definieren Sie wieder eine Lossfunktion und einen Optimizer. Denken Sie daran, dass wir eine Binary Klassifikation haben.
Da wir zwei Netze haben, die wir gemeinsam updaten wollen, können wir die Parameter der beiden Netze in einer Liste zusammenfassen und sie dem Optimizer zur Verfügung stellen.

In [ ]:
loss_funktion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(list(model.parameters()) + list(pred_ll.parameters()), lr =0.001) 

In [ ]:
for i in range(40):
    pred_ll.train()
    for input_, targets in train_loader:
        optimizer.zero_grad()
        rnn_output = model(input_)[1][0]
        output = pred_ll(rnn_output).flatten()
        
        loss = loss_funktion(output, targets)
        loss.backward()
        optimizer.step()
    
    pred_ll.eval()
    
    rnn_output = model(train_x)[1][0]    
    output = pred_ll(rnn_output).flatten()
    loss_train = loss_funktion(output, train_y)
    auc_train = roc_auc_score(train_y.numpy(),torch.sigmoid(output).detach().clone().numpy())
    
    rnn_output = model(test_x)[1][0]    
    output = pred_ll(rnn_output).flatten()
    loss_test = loss_funktion(output, test_y)
    auc_test = roc_auc_score(test_y.numpy(),torch.sigmoid(output).detach().clone().numpy())
    
    print("Training Loss: %.3f Training AUC: %.3f | Test Loss: %.3f Test AUC: %.3f"
        % (loss_train.item(), auc_train,loss_test.item(), auc_test ))


Sie können sehen, dass Sie mit einem RNN genaue Vorhersagen machen können. In der Realität funktionieren oft ECFP und klassische neuronale Netze jedoch besser. Insbesondere bei kleinen Datensätzen, da sie nicht so komplex sind. 

Zuletzt betrachten wir die gelernten Embeddings. Hierfür speichern wir die Weightmatrix der Embeddings Layer.

In [ ]:
embedding_weights = list(model[0].parameters())[0].detach().clone().numpy()
embedding_weights.round(2)

Eine Möglichkeit die Embeddings zu analysieren, ist die Ähnlichkeit verschiedener Token über die `cosine_similarity` vergleichen. Tokens mit ähnlicher Funktion sollten ähnliche Embeddings haben.

Als Beispiel berechnen wir die Ähnlichkeit der Embeddings von einem Stickstoff in einem aromatischen Ring (`n`).
Dafür finden wir im Dictionary welche Zahl zu `"n"` gehört, und damit auch den Index der Reihe in der Embedding Matrix.


In [ ]:
idx_n = dictionary["n"]
dictionary["n"]

Wir berechnen die Similarity von dieser Embedding zu allen andere Embeddings. Im Anschluß wird ein Barchart erstellt.

In [ ]:
similarity_N = cosine_similarity(embedding_weights[idx_n:idx_n+1,:],embedding_weights)[0]
labels = [x for x in dictionary]

In [ ]:
sorted_values=pd.DataFrame({"symbol": labels, "similarity":similarity_N}).sort_values("similarity", ascending =False)
sorted_values.plot.bar("symbol", "similarity")

Das Problem bei einem so kleinen Datensatz ist, dass die Embeddings extrem vom Datensatz abhängig sind. Dennoch lassen sich allgemeine Trends erkennen. `n` ist den aromatischen Atomen "o" oder "c" ähnlicher als den Atomen außerhalb eines aromatischen Rings `C`,`N` und `O`. Die genauen Embeddings können jedoch von Training zu Training extrem variieren.

Sie können auch andere Symbole vergleichen, indem Sie hier nachsehen welcher Wert einenm bestimmten Token zugeordnet ist:

`idx_n = dictionary["n"]`

Wählen Sie ein anderes Symbol.